### 1. top_n_feature(original_list, top_n)

feature imporatance에서 나온 순서대로 전체 특성을 입력하면

순서를 유지하면서 원핫인코딩으로 인해 생긴 _GD와 같은 것들을 없애고,

상위 top_n개의 특성 리스트와 나머지의 지워야할 리스트를 알려주는 함수

In [6]:
def top_n_feature(original_list, top_n):
    edit_list=[]
    for i in original_list:
        result=i.split('_')[0]
        edit_list.append(result)
    
    ordered_feature_list=[]
    seen=set()
    for i in edit_list:
        if i not in seen:
            ordered_feature_list.append(i)
            seen.add(i)
    
    total_list=[]        
    top_n_list=ordered_feature_list[:top_n]
    top_n_list.append('SalePrice')
    erase_list=ordered_feature_list[top_n:]
    total_list.append(f'top_n_list({top_n})'),
    total_list.append(top_n_list)
    total_list.append('erase_list')
    total_list.append(erase_list)
    # 타겟변수인 SalePrice도 추가
    for i in total_list:
        print(i)
    
    return total_list
    

### 2. model_val(model, X_train, y_train)

교차검증 결과를 뽑아주는 함수 (RMSE 평균, 표준편차)가 출력되도록 설정되어있음


In [7]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error
def model_val(model, X_train, y_train):
    scoring = ['neg_mean_squared_error', 'neg_mean_absolute_error']
    cv_results = cross_validate(model, X_train, y_train, scoring=scoring, cv=5, return_train_score=True)
    
    for fold in range(5):
        rmse = np.sqrt(-cv_results['test_neg_mean_squared_error'][fold])
        mse = -cv_results['test_neg_mean_squared_error'][fold]
        score = -cv_results['test_neg_mean_absolute_error'][fold]
        print(f"CV Fold {fold + 1} - Score: {score:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}")
    
    print("RMSE 평균:", np.mean(np.sqrt(-cv_results['test_neg_mean_squared_error'])))
    print("RMSE 표준편차:", np.std(np.sqrt(-cv_results['test_neg_mean_squared_error'])))

### 3. feature_importance(model, X_train, top_n=30)

완성된 모델의 Feature를 feature_importance 순서대로 출력하고, 시각화하는 함수

In [8]:
import matplotlib.pyplot as plt
def feature_importance(model, X_train, top_n=30):
    importance=model.feature_importances_
    feature_names = X_train.columns
    # feature importance를 기준으로 내림차순 정렬
    sorted_idx = importance.argsort()[::-1]
    sorted_names = [feature_names[i] for i in sorted_idx]
    sorted_importance = importance[sorted_idx]

    # 상위 top_n개만 선택
    sorted_names = sorted_names[:top_n]
    sorted_importance = sorted_importance[:top_n]
    
    #특성이름 출력
    print(sorted_names)
   
    # 바 차트로 시각화
    plt.figure(figsize=(5, 5))
    plt.barh(sorted_names, sorted_importance)
    plt.xlabel('Feature Importance')
    plt.ylabel('Features')
    plt.title('Feature Importance')
    plt.show()

### 4. visualize_model_predictions(model, X_test, y_test)

완성된 모델로 test set을 예측하고, 예측값과 실제값을 산점도로 시각화하는 함수

In [9]:
import matplotlib.pyplot as plt
import numpy as np

def visualize_model_predictions(model, X_test, y_test):
    y_pred = model.predict(X_test)
    slope, intercept = np.polyfit(y_pred, y_test, 1)

    plt.figure(figsize=(5, 5))

    # 실제값과 예측값의 산점도
    plt.scatter(y_test, y_pred, label='Test Samples')

    # 모델 예측 결과 선
    plt.plot(y_pred, slope * y_pred + intercept, color='red', label="실제 모델")
    
    #기준선
    plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], '-', color='green',label="이상적인 모델")


    plt.title("실제값, 예측값 비교")
    plt.xlabel("실제값")
    plt.ylabel("예측값")
    plt.legend(loc='lower right')
    plt.show()


### 5. 계수 중요도 시각화 (회귀 모델일 경우 활용)

feature importance와 유사하게 회귀모델의 계수를 시각화하는 함수

In [10]:
def Regression_coef(model, X_train, top_n=30):
    # 각 특성의 계수(coefficient)
    coefficients = model.coef_

    # 특성 이름
    feature_names = X_train.columns

    # 계수의 절댓값으로 내림차순 정렬
    sorted_idx = abs(coefficients).argsort()[::-1]
    sorted_names = [feature_names[i] for i in sorted_idx]
    sorted_coefficients = coefficients[sorted_idx]

    # 바 차트로 시각화 (상위 30개만)
    plt.figure(figsize=(5, 5))
    plt.barh(sorted_names[:top_n], sorted_coefficients[:top_n])
    plt.xlabel('계수값')
    plt.ylabel('특성')
    plt.title('계수')
    plt.show()